In [17]:
import torch
import matplotlib.pyplot as plt

## 1. hi?hello!!

In [2]:
txt = list('hi?hello!!')*100 
txt_x = txt[:-1]
txt_y = txt[1:]

In [3]:
txt_x[:5], txt_y[:5]

(['h', 'i', '?', 'h', 'e'], ['i', '?', 'h', 'e', 'l'])

`txt_x`와 `txt_y`를 이용하여 아래와 같은 순서로 다음문자를 예측하고 싶은 신경망을 설계하고 싶다.

`h` $\to$ `i` $\to$ `?` $\to$ `h` $\to$ `e` $\to$ `l` $\to$ `l` $\to$ `o`
$\to$ `!` $\to$ `!`
$\to$ `h` $\to$ `i` $\to$ `?` $\to$ `h` $\to$ `e` 
 

***(1)-(6)*** 의 풀이에 공통적으로 필요한 과정 정리

In [4]:
def f(txt,mapping):
    return [mapping[key] for key in txt] 

sig = torch.nn.Sigmoid()
soft = torch.nn.Softmax(dim=1)
tanh = torch.nn.Tanh()
mapping = {'!':0, '?':1,'h':2,'i':3,'e':4,'l':5,'o':6} 
x= torch.nn.functional.one_hot(torch.tensor(f(txt_x,mapping))).float().to("cuda:0")
y= torch.nn.functional.one_hot(torch.tensor(f(txt_y,mapping))).float().to("cuda:0")

`(1)` `torch.nn.RNN()`을 이용하여 다음 문자를 예측하는 신경망을 설계하고 학습하라.

In [109]:
torch.manual_seed(42)
rnn = torch.nn.RNN(7, 4).to("cuda:0")
cook = torch.nn.Linear(4, 7).to("cuda:0")

loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(list(rnn.parameters()) + list(cook.parameters()), lr = 0.1)

#---#
for epoc in range(100) :
    h, _ = rnn(x)
    netout = cook(h)
    loss = loss_fn(netout, y)
    loss.backward()
    optimizr.step()
    optimizr.zero_grad()

In [110]:
h, _ = rnn(x)
netout = cook(h)

yhat = torch.nn.functional.softmax(netout, dim = 1).argmax(axis = 1)

In [111]:
inv_mapping = {v:k for v, k in zip(mapping.values(), mapping.keys())}
[inv_mapping[i.item()] for i in yhat][:20]

['i',
 '?',
 'h',
 'e',
 'l',
 'l',
 'o',
 '!',
 '!',
 'h',
 'i',
 '?',
 'h',
 'e',
 'l',
 'l',
 'o',
 '!',
 '!',
 'h']

`(2)` `torch.nn.RNNCell()`을 이용하여 다음 문자를 예측하는 신경망을 설계하고 학습하라.

In [115]:
torch.manual_seed(42)
rnncell = torch.nn.RNNCell(7, 4).to("cuda:0")
linr = torch.nn.Linear(4, 7).to("cuda:0")
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(list(rnncell.parameters()) + list(linr.parameters()), lr = 0.1)

#---#
L = len(x)
for epoc in range(100) :
    ht = torch.zeros(4).to("cuda:0")
    loss = 0

    for t in range(L) :
        ht = rnncell(x[t], ht)
        ot = linr(ht)
        
        loss += loss_fn(ot, y[t])

    loss = loss/L
    loss.backward()
    
    optimizr.step()
    optimizr.zero_grad()

In [116]:
h = torch.zeros((L, 4)).to("cuda:0")
h0 = torch.zeros(4).to("cuda:0")
h[0] = rnncell(x[0], h0)

for t in range(1, L) :
    h[t] = rnncell(x[t], h[t-1])

In [117]:
yhat = soft(linr(h))
yhat

tensor([[9.8811e-04, 4.8676e-05, 5.8596e-03,  ..., 6.1763e-02, 5.9356e-03,
         5.9702e-05],
        [1.4041e-02, 9.7211e-01, 2.4805e-04,  ..., 8.9972e-10, 1.3568e-02,
         2.0693e-06],
        [5.0399e-06, 3.7461e-04, 9.9567e-01,  ..., 6.5292e-04, 7.9580e-05,
         2.6547e-03],
        ...,
        [4.7723e-03, 2.3609e-06, 2.5208e-04,  ..., 2.0636e-03, 2.9616e-04,
         9.9261e-01],
        [9.9584e-01, 3.5457e-04, 5.0169e-05,  ..., 1.5502e-06, 1.3792e-04,
         3.6100e-03],
        [9.9120e-01, 4.4400e-03, 2.6194e-03,  ..., 7.3197e-07, 1.7851e-05,
         1.6988e-03]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

`(3)` `torch.nn.Module`을 상속받은 클래스를 정의하고 (2)의 결과와 동일한 적합값이 나오는 신경망을 설계한 뒤 학습하라. (초기값을 적절하게 설정할 것)

In [118]:
class rNNCell(torch.nn.Module) :
    def __init__(self) :
        super().__init__()
        self.i2h = torch.nn.Linear(7, 4)
        self.h2h = torch.nn.Linear(4, 4)
        self.tanh = torch.nn.Tanh()

    def forward(self, xt, ht) :
        ht = self.tanh(self.i2h(xt) + self.h2h(ht))

        return ht

In [119]:
torch.manual_seed(42)
rnncell = torch.nn.RNNCell(7, 4).to("cuda:0")
linr = torch.nn.Linear(4, 7).to("cuda:0")

rnncell2 = rNNCell()
rnncell2.i2h.weight.data = rnncell.weight_ih.data
rnncell2.h2h.weight.data = rnncell.weight_hh.data
rnncell2.i2h.bias.data = rnncell.bias_ih.data
rnncell2.h2h.bias.data = rnncell.bias_hh.data

loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(list(rnncell2.parameters()) + list(linr.parameters()), lr = 0.1)

#---#
L = len(x)
for epoc in range(100) :
    ht = torch.zeros(4).to("cuda:0")
    loss = 0

    for t in range(L) :
        ht = rnncell(x[t], ht)
        ot = linr(ht)
        
        loss += loss_fn(ot, y[t])

    loss = loss/L
    loss.backward()
    
    optimizr.step()
    optimizr.zero_grad()

In [120]:
h = torch.zeros((L, 4)).to("cuda:0")
h0 = torch.zeros(4).to("cuda:0")
h[0] = rnncell(x[0], h0)

for t in range(1, L) :
    h[t] = rnncell(x[t], h[t-1])

In [121]:
yhat = soft(linr(h))
yhat

tensor([[4.0261e-02, 4.3250e-05, 1.4219e-04,  ..., 4.2759e-01, 4.0815e-02,
         1.8438e-03],
        [4.0991e-02, 7.0688e-01, 6.0200e-02,  ..., 9.9645e-05, 1.8883e-01,
         5.9180e-04],
        [5.3898e-02, 1.1380e-01, 6.0440e-01,  ..., 5.8943e-04, 1.3457e-01,
         9.1508e-02],
        ...,
        [1.3807e-01, 2.1008e-03, 1.7078e-01,  ..., 1.5848e-02, 3.9910e-02,
         6.2809e-01],
        [8.9304e-01, 1.6627e-04, 4.7126e-04,  ..., 7.7563e-03, 8.3060e-02,
         1.4001e-02],
        [6.5835e-01, 1.3594e-02, 2.2517e-01,  ..., 1.0799e-02, 1.4963e-02,
         5.3452e-02]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [123]:
n1 = torch.nn.Embedding(2, 1)
n2 = torch.nn.Linear(1, 1)

## 2. O/X

(1) LSTM은 RNN보다 장기기억에 유리하다. (True)

(2) torch.nn.Embedding(num_embeddings=2,embedding_dim=1)와 torch.nn.Linear(in_features=1,out_features=1)의 학습가능한 파라메터수는 같다. (True)

(3) 아래와 같은 네트워크를 고려하자.

net = torch.nn.Linear(1,1)

차원이 (n,1) 인 임의의 텐서에 대하여 net(x)와 net.forward(x)의 출력결과는 같다. (True)

(4) 아래와 같이 a,b,c,d 가 반복되는 문자열이 반복되는 자료에서 다음문자열을 맞추는 과업을 수행하기 위해서는 반드시 순환신경망의 형태로 설계해야만 한다. (False) –> 오타로인하여 문제삭제

a,b,c,d,a,b,c,d,...
(5) RNN 혹은 LSTM 으로 신경망을 설계할 시 손실함수는 항상 torch.nn.CrossEntropyLoss 를 사용해야 한다. (False)

(1) Dropout Layer를 추가하면 layer의 추가로 인해 모형이 더 복잡해지며 그 결과 모형이 과적합될 수 있다.

(2) CNN에서 torch.nn.Conv2d() 로 만들어지는 레이어는 입력이미지의 특징을 추출하는 역할을 하며, 훈련과정에서 학습되지 않는다.

(3) MF-based 추천 시스템에서는 사용자의 성향과 항목의 성향을 내적하여 궁합을 계산한다.

(4) LSTM은 RNN과 비교할때 장기기억에 강점을 보인다.

(5) 강화 학습에서 에이전트가 현재 상태에서 취하는 행동은 ’리워드(Reward)’라고 한다.